<a href="https://colab.research.google.com/github/hwuachen/fa_assistant/blob/main/notebook/OpenAI_Function_Calling_DB_Career.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Function Calling


## Setup

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive/')
!pip install -r "/content/drive/MyDrive/Colab Notebooks/fa_assistant/requirements.txt"

In [ ]:

!pip install -qU \
  "openai" \
  "diffusers" \
  "transformers" \
  "accelerate" \
  "scipy" \
  "safetensors" \
  "xformers"


In [ ]:
import os, config, requests
import gradio as gr
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import openai

# Config

In [ ]:
LLM_COMPLETION_MODEL = "gpt-3.5-turbo-0613"
from getpass import getpass
# openai.api_key = config.OPENAI_API_KEY
openai.api_key = getpass()

# DB Career Search Criteria - https://careers.db.com/professionals/search-roles/#/professional/results/

In [ ]:
import json
import pandas as pd

def load_lookup_table(tablename):
  # Specify the file path
  file_path = f"{tablename}.json"
  print(file_path)

  # Open the file and load the JSON data
  with open(file_path, 'r') as file:
    data = json.load(file)

  # Parse JSON: string -> json data
  # data = json.loads(json_data)

  # Extract items
  items = data['lookup']['items']

  # Create a data frame
  df = pd.DataFrame(items)

  # # Rename columns
  # df.rename(columns={'id': 'ID', 'label': 'Label'}, inplace=True)
  return df

# Access the JSON data
# Example: printing the loaded JSON data
# df = load_lookup_table("career_level")
# print(df)


# Extract Label values to string
def extract_label_values_tostring(tablename):
  df = load_lookup_table(tablename)
  label_string = ', '.join(item for item in df['label'])
  return (label_string)

# Access all the label string
# Example: printing all the label string
# career_levels = extract_label_values_tostring("career_level")
# print(career_levels)

# Access a list of all citys
citynames = load_lookup_table("cityname")['label'].tolist()
print(citynames)

# Access a list of all career_levels
career_levels = load_lookup_table("career_level")['label'].tolist()
print(career_levels)


In [ ]:
def get_city_code(city):
  df = load_lookup_table("cityname")
  citycode = df.loc[df['label'] == city].values[0]
  return citycode[0]

# example
print(get_city_code("New York"))

def get_position_schedule_code(career_level):
  df = load_lookup_table("career_level")
  code = df.loc[df['label'] == career_level].values[0]
  return code[0]

# example
print(get_position_schedule_code("Analyst"))

In [ ]:
# --------------------------------------------------------------
#  DB Career API call Function, and wrap it as search_job_deutschebank
# --------------------------------------------------------------

import requests
import json
import pandas as pd

def search_job_deutschebank_call_api(career_level, cityname):
    citycode = get_city_code(cityname)
    positioncode = get_position_schedule_code(career_level)
    print(f'citycode = {citycode} positioncode = {positioncode} ')

    # API endpoint
    api_url = "https://api-deutschebank.beesite.de/search/"

    # Request payload
    payload = {
        "LanguageCode": "en",
        "SearchParameters": {
            "FirstItem": 1,
            "CountItem": 10000,
            "MatchedObjectDescriptor": [
                "Facet:ProfessionCategory",
                "Facet:UserArea.ProDivision",
                "Facet:Profession",
                "Facet:PositionLocation.CountrySubDivision",
                "Facet:PositionOfferingType.Code",
                "Facet:PositionSchedule.Code",
                "Facet:PositionLocation.City",
                "Facet:PositionLocation.Country",
                "Facet:JobCategory.Code",
                "Facet:CareerLevel.Code",
                "Facet:PositionHiringYear",
                "Facet:PositionFormattedDescription.Content",
                "PositionID",
                "PositionTitle",
                "PositionURI",
                "ScoreThreshold",
                "OrganizationName",
                "PositionFormattedDescription.Content",
                "PositionLocation.CountryName",
                "PositionLocation.CountrySubDivisionName",
                "PositionLocation.CityName",
                "PositionLocation.Longitude",
                "PositionLocation.Latitude",
                "PositionIndustry.Name",
                "JobCategory.Name",
                "CareerLevel.Name",
                "PositionSchedule.Name",
                "PositionOfferingType.Name",
                "PublicationStartDate",
                "UserArea.GradEduInstCountry",
                "PositionImport",
                "PositionHiringYear",
                "PositionID"
            ],
            "Sort": [
                {
                    "Criterion": "PublicationStartDate",
                    "Direction": "DESC"
                }
            ]
        },
        "SearchCriteria": [
            {
                "CriterionName": "PositionLocation.City",
                "CriterionValue": citycode
            },
            {
                "CriterionName": "PositionSchedule.Code",
                "CriterionValue": positioncode
            },
            {
                "CriterionName": "PositionFormattedDescription.Content"
            },
            None,
            None
        ]
    }

    # Convert payload to JSON string
    payload_json = json.dumps(payload)

    # Make API call
    response = requests.get(api_url, params={"data": payload_json})

    # Check if the request was successful
    if response.status_code == 200:
        # Extract and return the response JSON
        response_json = response.json()
        return response_json
    else:
        # Request was not successful, handle the error
        print("Error:", response.status_code)
        return None

# example
# response_json = search_job_deutschebank_call_api(career_level = "Analyst", city="New York")
# filtered_df = df_reponse['MatchedObjectDescriptor.PositionTitle'].str.contains("Analyst")
# df = pd.json_normalize(response_json['SearchResult']['SearchResultItems'])
# df.loc[0]['MatchedObjectDescriptor.CareerLevel'][0]['Name']

## DB Search

In [84]:
def search_job_deutschebank(career_level: str, cityname: str) -> str:
  """Searches for job positions at Deutsche Bank based on the specified career level and city name.

    Args:
        career_level (str): The desired career level for the job search.
        cityname (str): The name of the city where the job search is to be performed.

    Returns:
        str: A JSON string containing the details of the job positions that match the specified career level
        and city name. The JSON structure includes the matched object ID, position title, organization name,
        and URL for each job position.
  """

  print("search_job_deutschebank is called")
  response_json = search_job_deutschebank_call_api(career_level, cityname)

  # Convert JSON to DataFrame
  df = pd.json_normalize(response_json['SearchResult']['SearchResultItems'])

  df['inscope'] = 'N'
  df['url'] = ''
  for index, row in df.iterrows():
    df['url'][index] =  "https://careers.db.com/professionals/search-roles/#/professional/job/" + df['MatchedObjectId'][index]
    # if ("Vice President" in df['MatchedObjectDescriptor.CareerLevel'][index][0]['Name']):
    if ("Analyst" in df['MatchedObjectDescriptor.CareerLevel'][index][0]['Name']):
      print(df['MatchedObjectDescriptor.CareerLevel'][index][0]['Name'])
      df['inscope'][index] ='Y'

  result = df.loc[df['inscope'] == 'Y']
  df_result = result[['MatchedObjectId','MatchedObjectDescriptor.PositionTitle', 'MatchedObjectDescriptor.OrganizationName', 'url']]
  df_result1 = df_result.rename(columns={"MatchedObjectId": "jobId", "MatchedObjectDescriptor.PositionTitle": "PositionTitle", \
                            'MatchedObjectDescriptor.OrganizationName':"OrganizationName"})

  json_data = df_result1.to_json(orient='records')
  return json.dumps(json_data)

# example
result = search_job_deutschebank('Analyst', 'New York')
result


search_job_deutschebank is called
cityname.json
career_level.json
citycode = 135 positioncode = 1 
Analyst
Analyst
Analyst
Analyst
Analyst


<ipython-input-84-292008997db6>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['url'][index] =  "https://careers.db.com/professionals/search-roles/#/professional/job/" + df['MatchedObjectId'][index]
<ipython-input-84-292008997db6>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inscope'][index] ='Y'


'"[{\\"jobId\\":\\"44551\\",\\"PositionTitle\\":\\"Transaction Coordinator - Analyst\\",\\"OrganizationName\\":\\"New York\\",\\"url\\":\\"https:\\\\/\\\\/careers.db.com\\\\/professionals\\\\/search-roles\\\\/#\\\\/professional\\\\/job\\\\/44551\\"},{\\"jobId\\":\\"44136\\",\\"PositionTitle\\":\\"Capital Markets Specialist - ETF\\",\\"OrganizationName\\":\\"New York\\",\\"url\\":\\"https:\\\\/\\\\/careers.db.com\\\\/professionals\\\\/search-roles\\\\/#\\\\/professional\\\\/job\\\\/44136\\"},{\\"jobId\\":\\"43528\\",\\"PositionTitle\\":\\"Investment Grade Trader\\",\\"OrganizationName\\":\\"New York\\",\\"url\\":\\"https:\\\\/\\\\/careers.db.com\\\\/professionals\\\\/search-roles\\\\/#\\\\/professional\\\\/job\\\\/43528\\"},{\\"jobId\\":\\"42320\\",\\"PositionTitle\\":\\"Reward &amp; Recognition Analyst\\",\\"OrganizationName\\":\\"New York\\",\\"url\\":\\"https:\\\\/\\\\/careers.db.com\\\\/professionals\\\\/search-roles\\\\/#\\\\/professional\\\\/job\\\\/42320\\"},{\\"jobId\\":\\"36547

# OpenAI

In [85]:
# --------------------------------------------------------------
# Ask ChatGPT a Question
# --------------------------------------------------------------

completion = openai.ChatCompletion.create(
    model=LLM_COMPLETION_MODEL,
    messages=[
        {
            "role": "user",
            "content": "I am looking for Analyst job in Deutsche Bank. Is there avaialble Analyst position in New York?",
        },
    ],
)
output = completion.choices[0].message.content
print(output)

# output:
# I'm sorry, but as an AI language model, I don't have access to real-time job listings or information regarding specific job vacancies at Deutsche Bank. I recommend visiting the Deutsche Bank careers website or utilizing other job search platforms to check for available positions in New York.



I'm sorry, but as an AI language model, I don't have real-time access to job listings or current openings. However, Deutsche Bank does have a presence in New York, so it's possible that there might be available Analyst positions. I would recommend visiting the Deutsche Bank careers website or other job search platforms to find the most up-to-date information on job openings in New York.


In [86]:
# --------------------------------------------------------------
# Use OpenAI’s Function Calling Feature
# --------------------------------------------------------------

function_descriptions = [
    {
        "name": "search_job_deutschebank",
        "description": "Find job position at Deutschebank",
        "parameters": {
            "type": "object",
            "properties": {
                "career_level": {
                    "type": "string",
                    "description": "The career level of the job, e.g. Analyst, Associate, Director ,Vice President",
                },
                "cityname": {
                    "type": "string",
                    "description": "The city where the job is located, e.g. New York, London, Sydney, Mumbai",
                },
            },
            "required": ["career_level", "cityname"],
        },
    }
]

In [87]:
user_prompt = "I am looking for Analyst job in Deutsche Bank. Is there avaialble Analyst position in Cary?"

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": user_prompt}],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
)

# It automatically fills the arguments with correct info based on the prompt
# Note: the function does not exist yet

output = completion.choices[0].message
print(output)


{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "search_job_deutschebank",
    "arguments": "{\n  \"career_level\": \"Analyst\",\n  \"cityname\": \"Cary\"\n}"
  }
}


In [88]:
completion.choices

[<OpenAIObject at 0x7f74fca94950> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": null,
     "function_call": {
       "name": "search_job_deutschebank",
       "arguments": "{\n  \"career_level\": \"Analyst\",\n  \"cityname\": \"Cary\"\n}"
     }
   },
   "finish_reason": "function_call"
 }]

In [89]:
output = completion.choices[0].message
print(output)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "search_job_deutschebank",
    "arguments": "{\n  \"career_level\": \"Analyst\",\n  \"cityname\": \"Cary\"\n}"
  }
}


In [90]:
# Use the LLM output to convert llm unstructure text to structure param, then call the function
# The json.loads function converts the string to a Python object

career_level = json.loads(output.function_call.arguments).get("career_level")
cityname = json.loads(output.function_call.arguments).get("cityname")
params = json.loads(output.function_call.arguments)
type(params)

print(career_level)
print(cityname)
print(params)

# Call the function with arguments

chosen_function = eval(output.function_call.name)
print(f"chosen_function = {chosen_function}")

jobs = chosen_function(**params)
print(jobs)

Analyst
Cary
{'career_level': 'Analyst', 'cityname': 'Cary'}
chosen_function = <function search_job_deutschebank at 0x7f74fc82b1c0>
search_job_deutschebank is called
cityname.json
career_level.json
citycode = 119 positioncode = 1 
Analyst
Analyst
"[{\"jobId\":\"39696\",\"PositionTitle\":\"Java Developer - Analyst\",\"OrganizationName\":\"Cary\",\"url\":\"https:\\/\\/careers.db.com\\/professionals\\/search-roles\\/#\\/professional\\/job\\/39696\"},{\"jobId\":\"39724\",\"PositionTitle\":\"Java Engineer - Analyst\",\"OrganizationName\":\"Cary\",\"url\":\"https:\\/\\/careers.db.com\\/professionals\\/search-roles\\/#\\/professional\\/job\\/39724\"}]"


<ipython-input-84-292008997db6>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['url'][index] =  "https://careers.db.com/professionals/search-roles/#/professional/job/" + df['MatchedObjectId'][index]
<ipython-input-84-292008997db6>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inscope'][index] ='Y'


In [91]:
jobs

'"[{\\"jobId\\":\\"39696\\",\\"PositionTitle\\":\\"Java Developer - Analyst\\",\\"OrganizationName\\":\\"Cary\\",\\"url\\":\\"https:\\\\/\\\\/careers.db.com\\\\/professionals\\\\/search-roles\\\\/#\\\\/professional\\\\/job\\\\/39696\\"},{\\"jobId\\":\\"39724\\",\\"PositionTitle\\":\\"Java Engineer - Analyst\\",\\"OrganizationName\\":\\"Cary\\",\\"url\\":\\"https:\\\\/\\\\/careers.db.com\\\\/professionals\\\\/search-roles\\\\/#\\\\/professional\\\\/job\\\\/39724\\"}]"'

In [92]:
# --------------------------------------------------------------
# Add function result to the prompt for a final answer
# --------------------------------------------------------------

# The key is to add the function output back to the messages with role: function
second_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "function", "name": output.function_call.name, "content": jobs},
    ],
    functions=function_descriptions,
)
response = second_completion.choices[0].message.content
print(response)

Yes, there are Analyst positions available in Cary at Deutsche Bank. Here are two positions that you can consider:

1. Position: Java Developer - Analyst
   Location: Cary
   [Link to job posting](https://careers.db.com/professionals/search-roles/#/professional/job/39696)

2. Position: Java Engineer - Analyst
   Location: Cary
   [Link to job posting](https://careers.db.com/professionals/search-roles/#/professional/job/39724)

You can visit the respective job links for more information and to apply for the positions.


# using agent

In [ ]:
import inspect
import re


def type_mapping(dtype):
    if dtype == float:
        return "number"
    elif dtype == int:
        return "integer"
    elif dtype == str:
        return "string"
    else:
        return "string"


def extract_params(doc_str: str):
    # parse the docstring to get the descriptions for each parameter in dict format
    params_str = doc_str.split("\n\n")[1].split("\n")
    params = {}
    for line in params_str:
        param_match = re.findall(r'(?<=:param )\w+(?=:)', line)
        if param_match != []:
            param_name = param_match[0]
            desc_match = line.replace(f":param {param_name}:", "").strip()
            params[param_name] = desc_match
    return params


def func_to_json(func):
    # first we get function name
    func_name = func.__name__
    # then we get the function annotations
    argspec = inspect.getfullargspec(func)
    # get the function docstring
    func_doc = inspect.getdoc(func)
    # parse the docstring to get the description
    func_description = func_doc.split("\n\n")[0]
    print (func_description)

    # get params
    params = argspec.annotations
    if 'return' in params.keys():
        del params['return']
    # parse the docstring to get the descriptions for each parameter in dict format
    param_details = extract_params(func_doc)
    # attach parameter types to params
    print(params)
    print(param_details)
    for param_name in argspec.args:
        params[param_name] = {
            "description": param_details.get(param_name) or "",
            "type": type_mapping(argspec.annotations[param_name])
        }
    # calculate required parameters
    _required = argspec.args
    if inspect.getfullargspec(func).defaults:
        _required = [argspec.args[i] for i, a in enumerate(argspec.args) if
                     i + 1 not in inspect.getfullargspec(func).defaults]
    # then return everything in dict
    return {
        "name": func_name,
        "description": func_description,
        "parameters": {
            "type": "object",
            "properties": params
        },
        "required": _required
    }


In [ ]:
import json
from typing import Optional
# from funkagent import parser
# import parser
import openai

sys_msg = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussion on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""


class Agent:
    def __init__(
        self,
        openai_api_key: str,
        # model_name: str = 'gpt-4-0613',
        model_name: str = 'gpt-3.5-turbo-0613',
        functions: Optional[list] = None
    ):
        openai.api_key = openai_api_key
        self.model_name = model_name
        self.functions = self._parse_functions(functions)
        self.func_mapping = self._create_func_mapping(functions)
        self.chat_history = [{'role': 'system', 'content': sys_msg}]

    def _parse_functions(self, functions: Optional[list]) -> Optional[list]:
        if functions is None:
            return None
        # return [parser.func_to_json(func) for func in functions]
        return [func_to_json(func) for func in functions]

    def _create_func_mapping(self, functions: Optional[list]) -> dict:
        if functions is None:
            return {}
        return {func.__name__: func for func in functions}

    def _create_chat_completion(
        self, messages: list, use_functions: bool=True
    ) -> openai.ChatCompletion:
        if use_functions and self.functions:
            res = openai.ChatCompletion.create(
                model=self.model_name,
                messages=messages,
                functions=self.functions
            )
        else:
            res = openai.ChatCompletion.create(
                model=self.model_name,
                messages=messages
            )
        return res

    def _generate_response(self) -> openai.ChatCompletion:
        while True:
            print('.', end='')
            res = self._create_chat_completion(
                self.chat_history + self.internal_thoughts
            )
            finish_reason = res.choices[0].finish_reason

            if finish_reason == 'stop' or len(self.internal_thoughts) > 3:
                # create the final answer
                final_thought = self._final_thought_answer()
                final_res = self._create_chat_completion(
                    self.chat_history + [final_thought],
                    use_functions=False
                )
                return final_res
            elif finish_reason == 'function_call':
                self._handle_function_call(res)
            else:
                raise ValueError(f"Unexpected finish reason: {finish_reason}")

    def _handle_function_call(self, res: openai.ChatCompletion):
        self.internal_thoughts.append(res.choices[0].message.to_dict())
        func_name = res.choices[0].message.function_call.name
        args_str = res.choices[0].message.function_call.arguments
        result = self._call_function(func_name, args_str)
        res_msg = {'role': 'assistant', 'content': (f"The answer is {result}.")}
        self.internal_thoughts.append(res_msg)

    def _call_function(self, func_name: str, args_str: str):
        args = json.loads(args_str)
        func = self.func_mapping[func_name]
        res = func(**args)
        return res

    def _final_thought_answer(self):
        thoughts = ("To answer the question I will use these step by step instructions."
                    "\n\n")
        for thought in self.internal_thoughts:
            if 'function_call' in thought.keys():
                thoughts += (f"I will use the {thought['function_call']['name']} "
                             "function to calculate the answer with arguments "
                             + thought['function_call']['arguments'] + ".\n\n")
            else:
                thoughts += thought["content"] + "\n\n"
        self.final_thought = {
            'role': 'assistant',
            'content': (f"{thoughts} Based on the above, I will now answer the "
                        "question, this message will only be seen by me so answer with "
                        "the assumption with that the user has not seen this message.")
        }
        return self.final_thought

    def ask(self, query: str) -> openai.ChatCompletion:
        self.internal_thoughts = []
        self.chat_history.append({'role': 'user', 'content': query})
        res = self._generate_response()
        self.chat_history.append(res.choices[0].message.to_dict())
        return res

In [80]:
agent = Agent(
    openai_api_key=openai.api_key,
    functions=[search_job_deutschebank]
)


Searches for job positions at Deutsche Bank based on the specified career level and city name.
{'career_level': <class 'str'>, 'cityname': <class 'str'>}
{}


In [ ]:
agent.functions

In [81]:
user_prompt = "I am looking for Analyst job in Deutsche Bank. Is there avaialble Analyst position in Cary?"
output = agent.ask(user_prompt)
print(output.choices[0].message)

.search_job_deutschebank is called
cityname.json
career_level.json
citycode = 119 positioncode = 1 


<ipython-input-78-0a3b2d4b60f1>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['url'][index] =  "https://careers.db.com/professionals/search-roles/#/professional/job/" + df['MatchedObjectId'][index]
<ipython-input-78-0a3b2d4b60f1>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inscope'][index] ='Y'


Analyst
Analyst
.{
  "role": "assistant",
  "content": "I found two analyst positions at Deutsche Bank in Cary. The positions are for Java Developer - Analyst and Java Engineer - Analyst. You can find more information and apply for these positions on the Deutsche Bank careers website. Here are the links to the job postings:\n\n1. Java Developer - Analyst: [Link](https://careers.db.com/professionals/search-roles/#/professional/job/39696)\n2. Java Engineer - Analyst: [Link](https://careers.db.com/professionals/search-roles/#/professional/job/39724)\n\nPlease note that job availability is subject to change, so it's always a good idea to check the careers website for the most up-to-date information. Good luck with your job search!"
}


In [ ]:
print(output.choices[0].message)

In [ ]:
agent.chat_history

In [ ]:
user_prompt = "What is the Deutsche Bank Values & Diversity?"
output = agent.ask(user_prompt)
print(output.choices[0].message)

In [ ]:
user_prompt = "I am looking for Analyst job in Deutsche Bank in Cary. can you please send me a report in excel format"
output = agent.ask(user_prompt)
print(output.choices[0].message)

In [ ]:
def search_job_deutschebank(career_level: str, cityname: str) -> str:
  """Searches for job positions at Deutsche Bank based on the specified career level and city name.

    Args:
        career_level (str): The desired career level for the job search.
        cityname (str): The name of the city where the job search is to be performed.

    Returns:
        str: A JSON string containing the details of the job positions that match the specified career level
        and city name. The JSON structure includes the matched object ID, position title, organization name,
        and URL for each job position.
  """
  response_json = search_job_deutschebank_call_api(career_level, cityname)

  # Convert JSON to DataFrame
  df = pd.json_normalize(response_json['SearchResult']['SearchResultItems'])

  df['inscope'] = 'N'
  df['url'] = ''
  for index, row in df.iterrows():
    df['url'][index] =  "https://careers.db.com/professionals/search-roles/#/professional/job/" + df['MatchedObjectId'][index]
    # if ("Vice President" in df['MatchedObjectDescriptor.CareerLevel'][index][0]['Name']):
    if ("Analyst" in df['MatchedObjectDescriptor.CareerLevel'][index][0]['Name']):
      print(df['MatchedObjectDescriptor.CareerLevel'][index][0]['Name'])
      df['inscope'][index] ='Y'

  result = df.loc[df['inscope'] == 'Y']
  df_result = result[['MatchedObjectId','MatchedObjectDescriptor.PositionTitle', 'MatchedObjectDescriptor.OrganizationName', 'url']]
  json_data = df_result.to_json(orient='records')
  return json.dumps(json_data)

In [98]:
def submit_job_application(url: str, name: str, email_address: str)-> str:
    """Submit the job application to the specific url with name and email address

    Args:
        url (str): The desired job url link for the job submission.
        name (str): The name of the user
        email_address (str): The email address of the user w

    Returns:
        str: A JSON string containing the details of the job sumission for the name and email addresss. The JSON structure includes the name, email address and
        and URL for each job position.
    """
    print ('submit_job_application is called')
    return f"Submit the job application with {name} and {email_address} at {url}"

In [99]:
agent1 = Agent(
    openai_api_key=openai.api_key,
    functions=[search_job_deutschebank, submit_job_application]
)


Searches for job positions at Deutsche Bank based on the specified career level and city name.
{'career_level': <class 'str'>, 'cityname': <class 'str'>}
{}
Submit the job application to the specific url with name and email address
{'url': <class 'str'>, 'name': <class 'str'>, 'email_address': <class 'str'>}
{}


In [110]:
user_prompt = """
This is Jane Harris. I am an graduate who want to find job at Deutsche Bank.
First, I neeed to know the avaialble Analyst position in Cary
Then I want to submit job application to the position you found.
Please give me a confirmation after all of these are done.
"""

In [111]:
output = agent1.ask(user_prompt)
print(output.choices[0].message)

.search_job_deutschebank is called
cityname.json
career_level.json
citycode = 119 positioncode = 1 


<ipython-input-84-292008997db6>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['url'][index] =  "https://careers.db.com/professionals/search-roles/#/professional/job/" + df['MatchedObjectId'][index]
<ipython-input-84-292008997db6>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inscope'][index] ='Y'


Analyst
Analyst
.{
  "role": "assistant",
  "content": "I have found two available Analyst positions at Deutsche Bank in Cary:\n\n1. Java Developer - Analyst: [Link to Job Description](https://careers.db.com/professionals/search-roles/#/professional/job/39696)\n\n2. Java Engineer - Analyst: [Link to Job Description](https://careers.db.com/professionals/search-roles/#/professional/job/39724)\n\nTo apply for these positions, you can visit the links provided and review the job descriptions. If you find a position that matches your qualifications and interests, you can typically find an \"Apply Now\" or similar button on the page. Clicking this button will usually take you to an application form where you can input your information, upload your resume, and submit your application.\n\nIt's important to review the job requirements and tailor your application accordingly, highlighting relevant skills and experiences that make you a strong fit for the role. After submitting your application, y

In [112]:
user_prompt = """
Then I want to submit job application to the position you found.
Please give me a confirmation after all of these are done.
"""

In [113]:
output = agent1.ask(user_prompt)
print(output.choices[0].message)

.submit_job_application is called
.{
  "role": "assistant",
  "content": "I apologize for any confusion caused. However, as an AI language model, I don't have the capability to directly submit job applications or confirm actions in the physical world. My purpose is to provide information and assistance through text-based conversations.\n\nTo apply for the position you found at Deutsche Bank, you should follow the steps outlined on their website or application platform. Typically, you will need to fill out an online application form, providing your personal details, work experience, and any other required information. You may also be asked to upload your resume and cover letter.\n\nOnce you have completed and submitted your application, Deutsche Bank should send you a confirmation email to acknowledge receipt of your application. This confirmation email usually contains information about the next steps in the hiring process or a timeline for when you can expect to hear back from them.\n

In [115]:
agent1.chat_history

[{'role': 'system',
  'content': 'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussion on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a powerful system 

In [114]:
print(output.choices[0])

{
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "I apologize for any confusion caused. However, as an AI language model, I don't have the capability to directly submit job applications or confirm actions in the physical world. My purpose is to provide information and assistance through text-based conversations.\n\nTo apply for the position you found at Deutsche Bank, you should follow the steps outlined on their website or application platform. Typically, you will need to fill out an online application form, providing your personal details, work experience, and any other required information. You may also be asked to upload your resume and cover letter.\n\nOnce you have completed and submitted your application, Deutsche Bank should send you a confirmation email to acknowledge receipt of your application. This confirmation email usually contains information about the next steps in the hiring process or a timeline for when you can expect to hear back from them.\n\n

In [102]:
print(output.choices[0].message)

{
  "role": "assistant",
  "content": "I apologize for any confusion, but as an AI language model, I don't have direct access to real-time job information or the ability to submit job applications on your behalf. My previous response was based on information available at the time of training.\n\nTo find the most up-to-date information on available positions at Deutsche Bank in Cary, I recommend visiting the Deutsche Bank careers website. There, you can search for analyst positions in Cary and apply directly through the website. Once you have submitted your application, you should receive a confirmation email from Deutsche Bank.\n\nIt's important to note that the availability of specific positions may change over time, so I recommend regularly checking the careers website for the latest updates. Best of luck with your job search!"
}
